In [5]:
import pandas as pd

df_1 = pd.read_csv("Data\labelled_2_37.csv")
df_2 = pd.read_csv("Data\labelled_1_23.csv")
df_3 = pd.read_csv("Data\labelled_0_44.csv")
df_4 = pd.read_csv("Data\AGC_Data.csv")

df = pd.concat([df_3, df_2, df_4, df_1], axis=0)
df.to_csv("AGC_Combined.csv", index=False)

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer
)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
def engineer_time_series_features(df, speed_col='speed', timestamp_col='indo_time'):
    """
    Enhanced feature engineering specifically for time series anomaly detection.
    """
    features = pd.DataFrame(df[speed_col]).copy()
    features.columns = ['Speed']

    windows = {
        'ultra_short': 30,
        'short': 60,
        'medium': 300,
        'long': 900,
        'ultra_long': 1800
    }
    
    # 1. Enhanced Rolling Statistics
    for name, window in windows.items():
        min_periods = max(1, int(window * 0.1))
        
        # Basic statistics
        features[f'Speed_rolling_mean_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).mean()
        )
        features[f'Speed_rolling_std_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).std().fillna(0)
        )
        features[f'Speed_rolling_median_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).median()
        )
        
        # Advanced statistics
        features[f'Speed_rolling_skew_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).skew().fillna(0)
        )
        features[f'Speed_rolling_kurt_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).kurt().fillna(0)
        )
        
        # Range-based features
        rolling_min = df[speed_col].rolling(window=window, min_periods=min_periods).min()
        rolling_max = df[speed_col].rolling(window=window, min_periods=min_periods).max()
        features[f'Speed_rolling_range_{window}'] = rolling_max - rolling_min
        
        # Coefficient of variation
        features[f'Speed_rolling_cv_{window}'] = (
            features[f'Speed_rolling_std_{window}'] / 
            (features[f'Speed_rolling_mean_{window}'] + 1e-9)
        )
        
        # Z-score
        features[f'Speed_rolling_zscore_{window}'] = (
            (df[speed_col] - features[f'Speed_rolling_mean_{window}']) /
            (features[f'Speed_rolling_std_{window}'] + 1e-9)
        )
        
        # Mean-median difference (asymmetry indicator)
        features[f'Speed_mean_median_diff_{window}'] = (
            features[f'Speed_rolling_mean_{window}'] - features[f'Speed_rolling_median_{window}']
        )
        
        # Percentiles
        features[f'Speed_rolling_q10_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).quantile(0.1)
        )
        features[f'Speed_rolling_q90_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).quantile(0.9)
        )
        features[f'Speed_rolling_iqr_{window}'] = (
            df[speed_col].rolling(window=window, min_periods=min_periods).quantile(0.75) -
            df[speed_col].rolling(window=window, min_periods=min_periods).quantile(0.25)
        )
    
    # 2. Temporal Change Features
    for lag in [1, 5, 10, 30, 60, 120]:
        features[f'Speed_diff_{lag}'] = df[speed_col].diff(periods=lag).fillna(0)
        features[f'Speed_pct_change_{lag}'] = df[speed_col].pct_change(periods=lag).fillna(0)
        features[f'Speed_lag_{lag}'] = df[speed_col].shift(lag).fillna(df[speed_col].mean())
    
    # Acceleration and jerk
    features['Speed_acceleration'] = df[speed_col].diff().diff().fillna(0)
    features['Speed_jerk'] = df[speed_col].diff().diff().diff().fillna(0)
    
    # 3. Exponential Weighted Features (multiple spans)
    for span in [10, 30, 60, 120]:
        alpha = 2 / (span + 1)
        features[f'Speed_ewm_mean_span_{span}'] = (
            df[speed_col].ewm(span=span, adjust=False, min_periods=1).mean()
        )
        features[f'Speed_ewm_std_span_{span}'] = (
            df[speed_col].ewm(span=span, adjust=False, min_periods=1).std().fillna(0)
        )
        features[f'Speed_ewm_diff_span_{span}'] = (
            df[speed_col] - features[f'Speed_ewm_mean_span_{span}']
        )
    
    # 4. CUSUM Features (both positive and negative)
    for window in [60, 300]:
        mean_col = f'Speed_rolling_mean_{window}'
        # Positive deviations
        features[f'Speed_positive_deviation_{window}'] = np.maximum(
            0, df[speed_col] - features[mean_col]
        )
        features[f'Speed_cusum_positive_{window}'] = (
            features[f'Speed_positive_deviation_{window}']
            .rolling(window=window, min_periods=1)
            .sum()
        )
        
        # Negative deviations
        features[f'Speed_negative_deviation_{window}'] = np.maximum(
            0, features[mean_col] - df[speed_col]
        )
        features[f'Speed_cusum_negative_{window}'] = (
            features[f'Speed_negative_deviation_{window}']
            .rolling(window=window, min_periods=1)
            .sum()
        )
    
    # 5. Time-based Features (if timestamp available)
    if timestamp_col in df.columns:
        # Extract time components
        features['hour'] = pd.to_datetime(df[timestamp_col]).dt.hour
        features['day_of_week'] = pd.to_datetime(df[timestamp_col]).dt.dayofweek
        features['is_weekend'] = features['day_of_week'].isin([5, 6]).astype(int)
        
        # Cyclic encoding
        features['hour_sin'] = np.sin(2 * np.pi * features['hour'] / 24)
        features['hour_cos'] = np.cos(2 * np.pi * features['hour'] / 24)
        features['dow_sin'] = np.sin(2 * np.pi * features['day_of_week'] / 7)
        features['dow_cos'] = np.cos(2 * np.pi * features['day_of_week'] / 7)
    
    # 6. Entropy-based Features (measure of randomness)
    def rolling_entropy(series, window):
        def entropy(x):
            if len(x) < 2:
                return 0
            hist, _ = np.histogram(x, bins=10)
            hist = hist[hist > 0]
            if len(hist) == 0:
                return 0
            prob = hist / hist.sum()
            return -np.sum(prob * np.log2(prob + 1e-9))
        
        return series.rolling(window=window, min_periods=max(1, window//10)).apply(entropy, raw=True)
    
    for window in [60, 300]:
        features[f'Speed_entropy_{window}'] = rolling_entropy(df[speed_col], window).fillna(0)
    
    # 7. Autoregressive Features
    for i in range(1, 6):
        features[f'Speed_ar_{i}'] = df[speed_col].shift(i).fillna(df[speed_col].mean())
    
    # 8. Moving Average Crossover Features
    features['Speed_ma_crossover_short_medium'] = (
        (features['Speed_rolling_mean_60'] > features['Speed_rolling_mean_300']).astype(int)
    )
    features['Speed_ma_crossover_medium_long'] = (
        (features['Speed_rolling_mean_300'] > features['Speed_rolling_mean_900']).astype(int)
    )

    features = features.fillna(features.mean())
    
    return features

In [3]:
def create_custom_scorer():
    """
    Create a custom scorer that balances between recall for anomalies and overall accuracy.
    """
    def anomaly_aware_score(y_true, y_pred):
        f1_anomaly = f1_score(y_true, y_pred, pos_label=1, average='binary')
        bal_acc = balanced_accuracy_score(y_true, y_pred)
        return 0.7 * f1_anomaly + 0.3 * bal_acc
    
    return make_scorer(anomaly_aware_score)

In [4]:
def detect_anomaly_range_ml_enhanced(
    df,
    speed_col='speed',
    label_col='pred_label',
    timestamp_col='indo_time',
    random_state=42,
    save_path_prefix='anomaly_interval',
    use_hyperparameter_tuning=True,
    n_iter=50
):
    """
    Enhanced anomaly detection with hyperparameter tuning and advanced features.
    """
    print("Building enhanced features for time series anomaly detection...")
    X = engineer_time_series_features(df, speed_col, timestamp_col)
    y = df[label_col]
    
    print(f"Total features created: {X.shape[1]}")

    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=0.20, stratify=y, random_state=random_state
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=random_state
    )
    
    print(f"\nShapes → Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    print("Anomaly distribution in TRAIN:", y_train.value_counts(normalize=True).to_dict())
    print("Anomaly distribution in VALID:", y_val.value_counts(normalize=True).to_dict())
    print("Anomaly distribution in TEST:", y_test.value_counts(normalize=True).to_dict())

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    neg_count = y_train.value_counts().get(0, 0)
    pos_count = y_train.value_counts().get(1, 0)
    scale_pos_weight = (neg_count / pos_count) if (pos_count > 0) else 1
    
    if use_hyperparameter_tuning:
        print("\nPerforming hyperparameter tuning...")

        param_grid = {
            'n_estimators': [100, 200, 300, 500],
            'max_depth': [3, 5, 7, 10, 15, None],
            'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
            'min_child_weight': [1, 3, 5, 7, 10],
            'gamma': [0, 0.1, 0.2, 0.3, 0.5],
            'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
            'reg_alpha': [0, 0.001, 0.01, 0.1, 1],
            'reg_lambda': [0, 0.001, 0.01, 0.1, 1],
            'scale_pos_weight': [scale_pos_weight, scale_pos_weight * 0.8, scale_pos_weight * 1.2]
        }

        base_model = XGBClassifier(
            random_state=random_state,
            use_label_encoder=False,
            eval_metric='logloss',
            early_stopping_rounds=20,
            n_jobs=4
        )
   
        scorer = create_custom_scorer()

        tscv = TimeSeriesSplit(n_splits=3)
   
        random_search = RandomizedSearchCV(
            base_model,
            param_distributions=param_grid,
            n_iter=n_iter,
            scoring=scorer,
            cv=tscv,
            verbose=1,
            random_state=random_state,
            n_jobs=4
        )

        random_search.fit(
            X_train_scaled, y_train,
            eval_set=[(X_val_scaled, y_val)],
            verbose=False
        )
        
        print(f"\nBest parameters: {random_search.best_params_}")
        print(f"Best CV score: {random_search.best_score_:.4f}")
        
        model = random_search.best_estimator_
    else:
        model = XGBClassifier(
            n_estimators=200,
            max_depth=7,
            learning_rate=0.1,
            min_child_weight=3,
            gamma=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=random_state,
            use_label_encoder=False,
            eval_metric='logloss',
            scale_pos_weight=scale_pos_weight,
            n_jobs=4
        )
        model.fit(X_train_scaled, y_train)

    print("\n--- MODEL EVALUATION ON VALIDATION SPLIT ---")
    y_val_pred = model.predict(X_val_scaled)
    y_val_proba = model.predict_proba(X_val_scaled)[:, 1]
    
    print(f"Accuracy (Val): {accuracy_score(y_val, y_val_pred):.4f}")
    print(f"Balanced Accuracy (Val): {balanced_accuracy_score(y_val, y_val_pred):.4f}")
    print(f"F1 Score - Anomaly Class (Val): {f1_score(y_val, y_val_pred, pos_label=1):.4f}")
    try:
        print(f"ROC AUC (Val): {roc_auc_score(y_val, y_val_proba):.4f}")
    except ValueError:
        print("ROC AUC (Val): N/A")
    
    print("\nClassification Report (Val):")
    print(classification_report(y_val, y_val_pred))
    print("Confusion Matrix (Val):")
    print(confusion_matrix(y_val, y_val_pred))

    print("\n--- TOP 20 FEATURE IMPORTANCES ---")
    feature_importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False).head(20)
    print(feature_importances.to_string(index=False))

    plt.figure(figsize=(10, 8))
    top_features = feature_importances.head(15)
    plt.barh(range(len(top_features)), top_features['Importance'])
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.xlabel('Importance')
    plt.title('Top 15 Feature Importances')
    plt.tight_layout()
    plt.savefig(f"{save_path_prefix}_feature_importances.png")
    plt.close()

    print("\nPredicting on TEST split and analyzing anomaly intervals...")
    y_test_pred = model.predict(X_test_scaled)
    y_test_proba = model.predict_proba(X_test_scaled)[:, 1]

    print(f"\nTest Set Performance:")
    print(f"Accuracy (Test): {accuracy_score(y_test, y_test_pred):.4f}")
    print(f"Balanced Accuracy (Test): {balanced_accuracy_score(y_test, y_test_pred):.4f}")
    print(f"F1 Score - Anomaly Class (Test): {f1_score(y_test, y_test_pred, pos_label=1):.4f}")

    test_idx = X_test.index.values
    test_df = pd.DataFrame({
        'orig_idx': test_idx,
        'pred': y_test_pred,
        'proba': y_test_proba
    }).sort_values(by='orig_idx').reset_index(drop=True)

    intervals_idx = []
    in_anomaly = False
    start_idx = None
    
    for row in test_df.itertuples(index=False):
        orig_i = row.orig_idx
        label = row.pred
        
        if (label == 1) and (not in_anomaly):
            in_anomaly = True
            start_idx = orig_i
        elif (label == 0) and in_anomaly:
            intervals_idx.append((start_idx, prev_i))
            in_anomaly = False
        
        prev_i = orig_i
    
    if in_anomaly:
        intervals_idx.append((start_idx, prev_i))
    
    if not intervals_idx:
        print("No predicted anomaly intervals in TEST split.")
        return [], model, scaler

    intervals_ts = []
    for (s_idx, e_idx) in intervals_idx:
        s_ts = df.loc[s_idx, timestamp_col]
        e_ts = df.loc[e_idx, timestamp_col]
        intervals_ts.append((s_ts, e_ts))

    context_window = 60
    for i, (s_idx, e_idx) in enumerate(intervals_idx[:5]):
        start_context = max(0, s_idx - context_window)
        end_context = min(len(df) - 1, e_idx + context_window)
        
        df_seg = df.loc[start_context:end_context].copy()

        seg_indices = df_seg.index
        seg_probas = test_df[test_df['orig_idx'].isin(seg_indices)]['proba'].values
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), height_ratios=[2, 1])

        sc = ax1.scatter(
            df_seg[timestamp_col],
            df_seg[speed_col],
            c=df_seg[label_col],
            cmap='coolwarm',
            s=15,
            edgecolors='none',
            alpha=0.7
        )
        
        s_ts = df.loc[s_idx, timestamp_col]
        e_ts = df.loc[e_idx, timestamp_col]
        
        ax1.axvline(x=s_ts, color='green', linestyle='--', linewidth=1.2, label='Anomaly Start')
        ax1.axvline(x=e_ts, color='red', linestyle='--', linewidth=1.2, label='Anomaly End')
        ax1.set_ylabel('Speed')
        ax1.set_title(f'Test Interval {i+1}: {s_ts} → {e_ts}')
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)

        if len(seg_probas) > 0:
            ax2.plot(df_seg[timestamp_col][:len(seg_probas)], seg_probas, 
                    color='purple', linewidth=1.5, alpha=0.8)
            ax2.axhline(y=0.5, color='black', linestyle=':', alpha=0.5)
            ax2.fill_between(df_seg[timestamp_col][:len(seg_probas)], 
                           seg_probas, 0.5, where=(seg_probas > 0.5),
                           color='red', alpha=0.3, label='Anomaly Region')
        
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Anomaly Probability')
        ax2.set_ylim(-0.05, 1.05)
        ax2.grid(True, alpha=0.3)
        
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        filename = f"{save_path_prefix}_test_interval_{i+1}_enhanced.png"
        plt.savefig(filename, dpi=150)
        plt.close()
        print(f"Saved enhanced plot: {filename}")
    
    print(f"\nFinal Result: Found {len(intervals_ts)} anomaly ranges in TEST split.")
    
    return intervals_ts, model, scaler

In [ ]:
def save_anomaly_model(model, scaler, feature_columns, model_path='anomaly_detection_model.pkl'):
    """
    Save the trained model, scaler, and feature information.
    """
    model_data = {
        'model': model,
        'scaler': scaler,
        'feature_columns': feature_columns,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_params': model.get_params()
    }
    
    joblib.dump(model_data, model_path)
    print(f"Model saved successfully to: {model_path}")
    
    # Also save feature columns separately for reference
    with open('model_features.txt', 'w') as f:
        f.write(f"Model trained on {len(feature_columns)} features\n")
        f.write(f"Timestamp: {model_data['timestamp']}\n\n")
        f.write("Feature list:\n")
        for i, feature in enumerate(feature_columns, 1):
            f.write(f"{i}. {feature}\n")
    
    return model_path

In [ ]:
def load_anomaly_model(model_path='anomaly_detection_model.pkl'):
    """
    Load the saved model and associated data.
    """
    model_data = joblib.load(model_path)
    print(f"Model loaded from: {model_path}")
    print(f"Model was trained on: {model_data['timestamp']}")
    print(f"Number of features: {len(model_data['feature_columns'])}")
    
    return model_data['model'], model_data['scaler'], model_data['feature_columns']

In [ ]:
def prepare_new_data(file_path, speed_col='A2:MCPGSpeed', timestamp_col='Time_stamp'):
    """
    Load and prepare new data for prediction.
    """
    # Read the new data
    df = pd.read_csv(file_path)
    
    # Ensure timestamp is datetime
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    
    # Sort by timestamp
    df = df.sort_values(by=timestamp_col).reset_index(drop=True)
    
    # Rename columns to match training data format
    df_prepared = pd.DataFrame({
        'speed': df[speed_col],
        'indo_time': df[timestamp_col]
    })
    
    print(f"Loaded {len(df_prepared)} rows of new data")
    print(f"Date range: {df_prepared['indo_time'].min()} to {df_prepared['indo_time'].max()}")
    print(f"Speed range: {df_prepared['speed'].min():.1f} to {df_prepared['speed'].max():.1f}")
    
    return df_prepared

In [ ]:
if __name__ == "__main__":
    file_path = 'Data\AGC_Combined.csv'
    try:
        print("Loading data...")
        data_df = pd.read_csv(file_path, parse_dates=['indo_time'])
        print(f"Data loaded: {data_df.shape[0]} rows, {data_df.shape[1]} columns")

        intervals_ts, model, scaler = detect_anomaly_range_ml_enhanced(
            df=data_df,
            speed_col='speed',
            label_col='pred_label',
            timestamp_col='indo_time',
            random_state=42,
            save_path_prefix='anomaly_interval',
            use_hyperparameter_tuning=True,
            n_iter=10
        )
        
        if intervals_ts:
            print(f"\nDetected anomaly intervals:")
            for i, (start, end) in enumerate(intervals_ts[:10], 1):
                print(f"  {i}. {start} → {end}")
        
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

Loading data...
Data loaded: 246001 rows, 4 columns
Building enhanced features for time series anomaly detection...
Total features created: 117

Shapes → Train: (147600, 117), Val: (49200, 117), Test: (49201, 117)
Anomaly distribution in TRAIN: {0: 0.940860433604336, 1: 0.05913956639566396}
Anomaly distribution in VALID: {0: 0.9408739837398374, 1: 0.0591260162601626}
Anomaly distribution in TEST: {0: 0.9408751854637101, 1: 0.05912481453628991}

Performing hyperparameter tuning...
Fitting 3 folds for each of 10 candidates, totalling 30 fits

Best parameters: {'subsample': 0.7, 'scale_pos_weight': np.float64(15.909153396723566), 'reg_lambda': 0.01, 'reg_alpha': 0.1, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': None, 'learning_rate': 0.3, 'gamma': 0.1, 'colsample_bytree': 0.7}
Best CV score: 0.9971

--- MODEL EVALUATION ON VALIDATION SPLIT ---
Accuracy (Val): 0.9997
Balanced Accuracy (Val): 0.9990
F1 Score - Anomaly Class (Val): 0.9976
ROC AUC (Val): 1.0000

Classification Rep